In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import types

In [ ]:
def process(df):

    galprops = pd.read_csv("bin/sdss_galprops_v1.0.dat", delimiter=' ', names=('Mag_g', 'Mag_r', 'sigma_v', 'Dn4000', 'concentration', 'log_M_star'))
    all_data = pd.merge(df, galprops, left_index=True, right_index=True)

    bins = np.logspace(np.log10(min(all_data.M_halo)), np.log10(max(all_data.M_halo)), 100)
    labels = bins[0:len(bins)-1] # using bottom (or top?) value, not middle

    all_data['Mh_bin'] = pd.cut(x = all_data['M_halo'], bins = bins, labels = labels, include_lowest = True)
    
    centrals = all_data[all_data.index == all_data.igrp]
    means = centrals.groupby('Mh_bin').log_M_star.mean()
    scatter = centrals.groupby('Mh_bin').log_M_star.std()

    dataset = types.SimpleNamespace()
    dataset.all_data = all_data
    dataset.bins = bins
    dataset.labels = labels
    dataset.centrals = centrals
    dataset.means = means
    dataset.scatter = scatter

    return dataset

def plots(*frames):

    plt.figure()
    for f in frames:
        plt.errorbar(f.labels, f.means, yerr=f.scatter)
    plt.xscale('log')
    plt.xlabel('M_halo / h')
    plt.ylabel('log(M_star)')
    plt.draw()

    plt.figure()    
    for f in frames:
        plt.errorbar(f.labels, f.scatter)
    plt.xscale('log')
    plt.xlabel('M_halo / h')
    plt.ylabel('$\\sigma(\\log(M_\\star)$')
    plt.draw()

    """     
    plt.figure()
    for f in frames:
        plt.scatter(f.centrals.M_halo, f.centrals.L_gal, alpha=0.002)
    plt.loglog()
    plt.xlabel('M_halo / h')
    plt.ylabel('L_gal / $h^2$)')
    plt.draw() 
    """

    plt.figure()
    for f in frames:
        plt.hist(f.centrals.N_sat, np.arange(0,50,1), alpha=0.5)
    plt.loglog()
    plt.draw()


In [ ]:
df = pd.read_csv("bin/run_1.out", delimiter=' ', names=('RA', 'Dec', 'z', 'L_gal', 'V_max', 'P_sat', 'M_halo', 'N_sat', 'L_tot', 'igrp', 'unknown'))
data1 = process(df)
#df2 = pd.read_csv("bin/run_beta_off_1.out", delimiter=' ', names=('RA', 'Dec', 'z', 'L_gal', 'V_max', 'P_sat', 'M_halo', 'N_sat', 'L_tot', 'igrp', 'unknown'))
#data2 = process(df2)
#df3 = pd.read_csv("bin/run_wcen_beta_off_1.out", delimiter=' ', names=('RA', 'Dec', 'z', 'L_gal', 'V_max', 'P_sat', 'M_halo', 'N_sat', 'L_tot', 'igrp', 'unknown'))
#data3 = process(df3)

df4 = pd.read_csv("bin/run_all_off_1.out", delimiter=' ', names=('RA', 'Dec', 'z', 'L_gal', 'V_max', 'P_sat', 'M_halo', 'N_sat', 'L_tot', 'igrp', 'unknown'))
data4 = process(df4)

In [ ]:
plots(data1, data4)